In [1]:
import torch

In [8]:

def compute_accuracy(labels, output):

    
    # (batch_size, 2_class,time_frame,224, 224) ->  (batch_size, 224, 224)
    #output=output.squeeze(2)
    predicted = torch.argmax(output, dim=1)
    print(predicted)
    # Compare the predicted class with the true labels
    correct = (predicted == labels).sum().item()
    total = labels.numel()  # Total number of elements in labels
    
    accuracy = correct / total 

    return accuracy


In [48]:
def compute_metrics(labels, output):
    # (batch_size, 2_class, time_frame, 224, 224) -> (batch_size, 224, 224)
    predicted = torch.argmax(output, dim=1)
    
    # Flatten the labels and predictions to work with individual elements
    predicted = predicted.view(-1)
    labels = labels.view(-1)
    
    # True Positives (TP): Both predicted and true labels are 1
    TP = ((predicted == 1) & (labels == 1)).sum().item()
    
    # True Negatives (TN): Both predicted and true labels are 0
    TN = ((predicted == 0) & (labels == 0)).sum().item()
    
    # False Positives (FP): Predicted is 1, but true label is 0
    FP = ((predicted == 1) & (labels == 0)).sum().item()
    
    # False Negatives (FN): Predicted is 0, but true label is 1
    FN = ((predicted == 0) & (labels == 1)).sum().item()
    
    # Accuracy: (TP + TN) / Total
    total = labels.numel()
    accuracy = (TP + TN) / total
    # Precision: TP / (TP + FP)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    # Recall: TP / (TP + FN)
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    #miou_1=TP/(TP+FN+FP)
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    

    return accuracy, f1_score

In [52]:
def calculate_miou(target,output):


    eps=1e-6
    #output=output.squeeze(2)
    num_classes=output.shape[1]
    preds = torch.argmax(output, dim=1)

    # Flatten the tensors
    preds = preds.view(-1)  # Flatten predictions
    target = target.view(-1)  # Flatten target
    
    # Initialize intersection and union for each class
    intersection = torch.zeros(num_classes)
    union = torch.zeros(num_classes)

    for cls in range(num_classes):

        # Create binary masks for the current class
        pred_mask = (preds == cls).float()
        target_mask = (target == cls).float()
       
        # Calculate intersection and union
        intersection[cls] = (pred_mask * target_mask).sum()
        union[cls] = pred_mask.sum() + target_mask.sum() - intersection[cls]
        
    # Calculate IoU for each class
    iou = intersection / (union + eps)  # Add eps to avoid division by zero
    
    # Calculate mean IoU
    mean_iou = iou.mean().item()

    return mean_iou

In [63]:
import numpy as np
output=torch.tensor([[[[0.1,0.1],[0.1,0.3]],[[0.3,0.2],[0.5,0.4]]],[[[0.1,0.1],[0.1,0.1]],[[0.3,0.2],[0.5,0.4]]]])
labels_1=torch.tensor([[[1,0],[0,1]],[[0,0],[0,1]]])
labels_2=torch.tensor([[[1,0],[1,1]],[[0,0],[0,1]]])
labels_3=torch.tensor([[[0,0],[1,1]],[[0,0],[0,1]]])
output=output.reshape(2,2,2,2)
labels_1=labels_1.reshape(2,2,2)
labels_2=labels_2.reshape(2,2,2)
labels_3=labels_3.reshape(2,2,2)
#x=calculate_miou(labels, output)

acc_t=[]
miou_t=[]
f1_t=[]
for i in range(3):
    if i==0:
        labels=labels_1
    elif i==1:
        labels=labels_2
    elif i==2:
        labels=labels_3
    acc,f1=compute_metrics(labels,output)
    miou=calculate_miou(labels,output)
    acc_t.append(acc)
    f1_t.append(f1)
    miou_t.append(miou)
    print(acc,f1,miou)

acc=np.mean(acc_t)
f1=np.mean(f1_t)
miou=np.mean(miou_t)
print(acc,f1,miou)

0.375 0.5454545454545454 0.1874999850988388
0.5 0.6666666666666666 0.2499999701976776
0.375 0.5454545454545454 0.1874999850988388
0.4166666666666667 0.5858585858585857 0.2083333134651184
